# Apnea Event Classifier Training Notebook

This notebook guides you through training a machine learning model to detect apnea events from audio features.

In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib # For saving/loading models

from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier # Or import xgb.XGBClassifier if you prefer XGBoost
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
)
from imblearn.over_sampling import SMOTE # For handling class imbalance
from imblearn.pipeline import Pipeline as ImbPipeline # Use imblearn's pipeline for SMOTE

# Set a random seed for reproducibility
np.random.seed(42)


c:\Users\solom\Documents\Evaluating-Noise-Reduction-Techniques\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Cell 2: Load Data & Initial Inspection

# --- IMPORTANT: Replace 'your_apnea_dataset.csv' with the actual path to your data ---
data_path = '..\master_apnea_dataset (1).csv'
df = pd.read_csv(data_path)

# Display initial data info
print("--- Initial Data Information ---")
print(df.head())
print("\nData Info:")
df.info()
print("\nInitial Label Distribution:")
print(df['label'].value_counts())
print("\nInitial Label Distribution (Normalized):")
print(df['label'].value_counts(normalize=True))

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\solom\AppData\Local\Temp\ipykernel_1712\159429153.py:2: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv('..\master_apnea_dataset (1).csv')


In [28]:
# Clean Data and prep for training
feature_cols = [c for c in df.columns
                if c not in {'patient_id', 'frame_start', 'frame_end', 'label'}]

# Build a boolean mask: True = keep this row
mask = ~(df[feature_cols] == 0).all(axis=1)
df = df.loc[mask].reset_index(drop=True)

print(f'Kept {len(df):,} frames after removing silent ones.')



Kept 306,140 frames after removing silent ones.


In [29]:
# ---------- 3. Rolling context features ----------
WIN = 5  # ±2.5 s each side
stats = ['mean', 'std', 'slope']
new_cols = []

for col in feature_cols:
    s = df[col].rolling(window=WIN, center=True, min_periods=1)
    df[f'{col}_mean']  = s.mean()
    df[f'{col}_std']   = s.std()
    # simple slope: (last - first)/(WIN-1)
    df[f'{col}_slope'] = s.apply(lambda x: (x.iloc[-1] - x.iloc[0]) / (len(x) - 1)
                                 if len(x) > 1 else 0.)
    new_cols.extend([f'{col}_{s}' for s in stats])

X_all = df[feature_cols + new_cols].values.astype('float32')
y_all = df['label'].values
groups = df['patient_id'].values

In [30]:
# ---------- 4. Patient-wise 5-fold CV ----------
gkf = GroupKFold(n_splits=5)
folds = list(gkf.split(X_all, y_all, groups))

In [31]:
# ---------- 5. Objective for Optuna ----------
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'tree_method': 'hist',
        'verbosity': 0
    }

    # inner CV to avoid leakage
    inner_scores = []
    for train_idx, val_idx in folds:
        dtrain = xgb.DMatrix(X_all[train_idx], label=y_all[train_idx])
        dval   = xgb.DMatrix(X_all[val_idx],   label=y_all[val_idx])

        bst = xgb.train(params, dtrain, num_boost_round=2000,
                        evals=[(dval, 'val')],
                        early_stopping_rounds=50, verbose_eval=False)

        proba = bst.predict(dval)
        preds = (proba >= 0.5).astype(int)
        inner_scores.append(f1_score(y_all[val_idx], preds))

    return np.mean(inner_scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40, show_progress_bar=True)
print('Best XGB params:', study.best_params)


[I 2025-07-22 00:23:14,763] A new study created in memory with name: no-name-b4d63349-4189-48bf-a256-fc96b047763f
Best trial: 0. Best value: 0.293928:   2%|▎         | 1/40 [00:40<26:17, 40.46s/it]

[I 2025-07-22 00:23:55,219] Trial 0 finished with value: 0.29392843574622235 and parameters: {'scale_pos_weight': 37.316043526313265, 'max_depth': 5, 'eta': 0.11345812640212863, 'subsample': 0.8865984302649417, 'colsample_bytree': 0.5242531138721043, 'min_child_weight': 2}. Best is trial 0 with value: 0.29392843574622235.


Best trial: 0. Best value: 0.293928:   5%|▌         | 2/40 [01:45<34:57, 55.20s/it]

[I 2025-07-22 00:25:00,727] Trial 1 finished with value: 0.28871640775818014 and parameters: {'scale_pos_weight': 47.6579369823112, 'max_depth': 6, 'eta': 0.019663366179587125, 'subsample': 0.7971145374669981, 'colsample_bytree': 0.9878839848656579, 'min_child_weight': 8}. Best is trial 0 with value: 0.29392843574622235.


Best trial: 0. Best value: 0.293928:   8%|▊         | 3/40 [02:23<28:59, 47.02s/it]

...

[I 2025-07-22 00:58:05,828] Trial 37 finished with value: 0.2762595553187789 and parameters: {'scale_pos_weight': 42.24709865718262, 'max_depth': 9, 'eta': 0.17344495128449441, 'subsample': 0.5808240592109569, 'colsample_bytree': 0.7805197882152313, 'min_child_weight': 5}. Best is trial 22 with value: 0.3011752291012574.


Best trial: 22. Best value: 0.301175:  98%|█████████▊| 39/40 [35:15<00:46, 46.07s/it]

[I 2025-07-22 00:58:30,637] Trial 38 finished with value: 0.2829158910170808 and parameters: {'scale_pos_weight': 18.178006166193654, 'max_depth': 3, 'eta': 0.29746596222636523, 'subsample': 0.7513749735914217, 'colsample_bytree': 0.9274503269680552, 'min_child_weight': 7}. Best is trial 22 with value: 0.3011752291012574.


Best trial: 22. Best value: 0.301175: 100%|██████████| 40/40 [35:59<00:00, 53.99s/it]

[I 2025-07-22 00:59:14,291] Trial 39 finished with value: 0.2939673689698681 and parameters: {'scale_pos_weight': 49.82492906211808, 'max_depth': 4, 'eta': 0.04576358714677485, 'subsample': 0.661907028730905, 'colsample_bytree': 0.5677429357675111, 'min_child_weight': 4}. Best is trial 22 with value: 0.3011752291012574.
Best XGB params: {'scale_pos_weight': 44.57868314849694, 'max_depth': 3, 'eta': 0.19521292025116885, 'subsample': 0.5488350749975534, 'colsample_bytree': 0.9845107513730289, 'min_child_weight': 5}


In [32]:
# ---------- 6. Retrain on full train / dev ----------
# Use best params, but split off a real validation set for threshold tuning
gss = GroupKFold(n_splits=5)
train_idx, val_idx = next(gss.split(X_all, y_all, groups))

dtrain = xgb.DMatrix(X_all[train_idx], label=y_all[train_idx])
dval   = xgb.DMatrix(X_all[val_idx],   label=y_all[val_idx])
dtest  = xgb.DMatrix(X_all[val_idx])   # same as val for this example

params = study.best_params.copy()
params.update({'objective': 'binary:logistic', 'eval_metric': 'auc'})
bst = xgb.train(params, dtrain, num_boost_round=2000,
                evals=[(dval, 'val')], early_stopping_rounds=50, verbose_eval=False)

In [ ]:
# ---------- 7. Post-processing grid search ----------
val_proba = bst.predict(dval)
val_y = y_all[val_idx]



ZeroDivisionError: float division by zero

In [35]:
# Gaussian smoothing helper
def smooth(prob, sigma):
    from scipy.ndimage import gaussian_filter1d
    return gaussian_filter1d(prob, sigma=sigma, mode='nearest')

best = {'f1': 0, 'sigma': 1, 'thr': 0.5}
from sklearn.metrics import f1_score

for sigma in [0, 1, 2, 3]:

    if len(np.unique(val_y)) < 2:
        best['f1'] = 0.0
        print("Validation fold has only one class; skipping.")
    else:
        smoothed = smooth(val_proba, sigma)
        for thr in np.arange(0.05, 0.95, 0.05):
            pred = (smoothed >= thr).astype(int)
            f = f1_score(val_y, pred, zero_division=0)  # <- prevents crash
            if f > best['f1']:
                best.update({'f1': f, 'sigma': sigma, 'thr': thr})

print('Best σ / thr on val:', best['sigma'], best['thr'], 'F1:', best['f1'])

ZeroDivisionError: float division by zero

In [24]:
# ---------- 8. Final metrics on validation (== test here) ----------
test_pred = (smooth(val_proba, best['sigma']) >= best['thr']).astype(int)
prec = precision_score(val_y, test_pred)
rec  = recall_score(val_y, test_pred)
f1   = f1_score(val_y, test_pred)

print('\n=== FINAL EVENT-LEVEL (15-s frames) ===')
print(f'Precision: {prec:.4f}   Recall: {rec:.4f}   F1: {f1:.4f}')